In [150]:
import pandas as pd
import numpy as np 
import re
import random
import copy

In [151]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet1")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet3")
df['QUESTION_SHOULD_CONTAIN']= df['QUESTION_SHOULD_CONTAIN'].str.lower()
slot_df['Slot']= slot_df['Slot'].str.lower()
df['TARGET_QUESTION']= df['TARGET_QUESTION'].str.lower()
df["TARGET_QUESTION"] = df['TARGET_QUESTION'].str.replace('[\.\?\']','')
## Preprocessing slot_df dataframe
# Slot dataframe trimmed so that the Description is removed. 
slot_df_clean = slot_df.iloc[:,[0,2]]
# replace the Example column with an array of the values separated by a comma
slot_df_clean = slot_df_clean[slot_df_clean['Examples'].notnull()].copy()
slot_df_clean.loc[:,'Examples'] = slot_df_clean.loc[:,"Examples"].apply(lambda x: x.split(', '))
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_clean.Slot, slot_df_clean.Examples))
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]
# A Series of arrays representing the respective slots
slots = df.iloc[:,4].apply(lambda x: x.split(' '))
# The dictionary where key:Intent value:Question
target_dic = df.iloc[:,[0,1]]
# A dictionary of intents key: int value: intent
intent_dict = df.QUESTION_LABEL.to_dict()
slot_dict_boi = copy.deepcopy(slot_dict)
for key,value in slot_dict.items():
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot
def random_slot(slot):
    random_slot = random.randint(0, len(slot_dict[slot])-1)
#     print(slot_dict[slot])
#     print(slot_dict[slot][random_slot])
    return slot_dict_boi[slot][random_slot]

In [152]:
file_name_train = 'appliance_data\\appliance_utterances_train.xlsx'
file_name_test = 'appliance_data\\appliance_utterances_test.xlsx'
file_name_valid = 'appliance_data\\appliance_utterances_valid.xlsx'

In [153]:
df_train = pd.read_excel(file_name_train)
df_test = pd.read_excel(file_name_test)
df_valid = pd.read_excel(file_name_valid)

In [154]:
df_valid.loc[265,'question']

'how do you take off a @removeobject from the @appliance'

In [155]:
df_valid.drop([53,107,114,124,134,140,158,159,210,229,249,258,271,291,296,320,321,326,330,331,333,334,341,342,358,365,399,409,418,440,441,456,459,464,472,480,491,515,542,590],axis=0,inplace=True)
df_test.drop([62,72,90,111,114,120,124,159,205,212,224,229,231,258,261,265,270,306,308,311,320,328,346,358,377,403,408,409,413,423,425,443,445,451,456,458,464,493,496,498,515,523,527,530,543,585],axis=0,inplace=True)
df_valid.reset_index(drop=True,inplace=True)
df_test.reset_index(drop=True,inplace=True)

In [163]:
df_test.to_excel('appliance_data_reverse\\appliance_utterances_test2.xlsx')
df_valid.to_excel('appliance_data_reverse\\appliance_utterances_valid2.xlsx')

In [156]:
f_train = open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance5\\train','w')
f_test = open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance5\\test','w')
f_valid = open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance5\\valid','w')

In [157]:
def create_data_file(f,df):
    for i in range(df.shape[0]):
        for j in range(3):
            slots = df.loc[i,'question_should_contain'].split(' ')
            x = df.loc[i,'question']
            x = re.sub(slots[0], random_slot(slots[0]), x)
            x = re.sub(slots[1], random_slot(slots[1]), x)
            x = x.lstrip()
            x = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            x = ' '.join(x.split())
            f.write(x + " <=> " + df.loc[i,'intent'] + "\n")
            print(x + " <=> " + df.loc[i,'intent'] + "\n")
    f.close()

In [158]:
create_data_file(f_train,df_train)

will:O i:O be:O able:O to:O use:O polyester:B-cookware cup:I-cookware in:O the:O oven:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O polyester:B-cookware cup:I-cookware in:O the:O microwavawe:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O thermometer:B-cookware in:O the:O oven:B-appliance <=> appropriate

is:O the:O microwavawe:B-appliance glassware:B-cookware safe:O <=> appropriate

is:O the:O oven:B-appliance straw:B-cookware safe:O <=> appropriate

is:O the:O microwavawe:B-appliance polyester:B-cookware cardboard:I-cookware safe:O <=> appropriate

can:O i:O use:O the:O wicker:B-cookware for:O the:O microwavawe:B-appliance <=> appropriate

can:O i:O use:O the:O paper:B-cookware bags:I-cookware for:O the:O oven:B-appliance <=> appropriate

can:O i:O use:O the:O browning:B-cookware plate:I-cookware for:O the:O microwavawe:B-appliance <=> appropriate

will:O it:O be:O safe:O to:O use:O the:O stoneware:B-cookware on:O the:O oven:B-appliance <=> appropriate


hey:O i:O need:O to:O learn:O what:O does:O c-d0:B-infocode mean:O on:O my:O microwavawe:B-appliance <=> code

hey:O i:O need:O to:O learn:O what:O does:O c-A2:B-infocode mean:O on:O my:O microwavawe:B-appliance <=> code

what:O is:O the:O meaning:O of:O c-A2:B-infocode in:O my:O microwavawe:B-appliance <=> code

what:O is:O the:O meaning:O of:O c-20:B-infocode in:O my:O microwavawe:B-appliance <=> code

what:O is:O the:O meaning:O of:O c-F0:B-infocode in:O my:O oven:B-appliance <=> code

what:O can:O c-21:B-infocode mean:O in:O my:O microwavawe:B-appliance <=> code

what:O can:O c-21:B-infocode mean:O in:O my:O oven:B-appliance <=> code

what:O can:O c-d1:B-infocode mean:O in:O my:O oven:B-appliance <=> code

why:O does:O bAd-LInE:B-infocode tell:O me:O my:O oven:B-appliance <=> code

why:O does:O c-22:B-infocode tell:O me:O my:O oven:B-appliance <=> code

why:O does:O c-22:B-infocode tell:O me:O my:O oven:B-appliance <=> code

why:O does:O c-22:B-infocode tell:O me:O my:O microwavaw


please:O tell:O me:O if:O know:O what:O must:O i:O do:O to:O cook:O frozen:B-foodtype onion:I-foodtype rings:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

please:O tell:O me:O if:O know:O what:O must:O i:O do:O to:O cook:O frozen:B-foodtype hash:I-foodtype brown:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

please:O tell:O me:O if:O know:O what:O must:O i:O do:O to:O cook:O frozen:B-foodtype chicken:I-foodtype nugets:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

what:O would:O i:O have:O to:O do:O to:O cook:O breaded:B-foodtype onions:I-foodtype with:O the:O oven:B-appliance <=> cook

what:O would:O i:O have:O to:O do:O to:O cook:O breaded:B-foodtype asparagus:I-foodtype with:O the:O oven:B-appliance <=> cook

what:O would:O i:O have:O to:O do:O to:O cook:O salmon:B-foodtype steak:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

hey:O show:O me:O what:O must:O i:O do:O to:O cook:O breaded:B-foodtype onions:I-foodtype with:O the:O microwavaw

how:O can:O i:O use:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O microwavawe:B-appliance <=> on_off

how:O can:O i:O use:O the:O sound:B-onofffeature in:O the:O microwavawe:B-appliance <=> on_off

turn:O off:O now:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature and:O microwavawe:B-appliance <=> on_off

turn:O off:O now:O wifi:B-onofffeature and:O oven:B-appliance <=> on_off

turn:O off:O now:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature and:O microwavawe:B-appliance <=> on_off

turn:O off:O already:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature and:O oven:B-appliance <=> on_off

turn:O off:O already:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature and:O microwavawe:B-appliance <=> on_off

turn:O off:O already:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature and:O microwavawe:B-appliance <=> on_off

how:O can:O i:O disable:O my:O wifi:B-onofffeature in:O

In [159]:
create_data_file(f_test,df_test)

is:O glassware:B-cookware appropriate:O to:O use:O in:O the:O oven:B-appliance <=> appropriate

is:O wood:B-cookware appropriate:O to:O use:O in:O the:O microwavawe:B-appliance <=> appropriate

is:O porcelain:B-cookware appropriate:O to:O use:O in:O the:O oven:B-appliance <=> appropriate

should:O i:O use:O the:O polyester:B-cookware cup:I-cookware in:O my:O oven:B-appliance <=> appropriate

should:O i:O use:O the:O wood:B-cookware in:O my:O microwavawe:B-appliance <=> appropriate

should:O i:O use:O the:O porcelain:B-cookware in:O my:O oven:B-appliance <=> appropriate

is:O the:O stoneware:B-cookware safe:O for:O use:O with:O the:O oven:B-appliance <=> appropriate

is:O the:O paper:B-cookware safe:O for:O use:O with:O the:O microwavawe:B-appliance <=> appropriate

is:O the:O stoneware:B-cookware safe:O for:O use:O with:O the:O oven:B-appliance <=> appropriate

is:O the:O polyester:B-cookware cardboard:I-cookware safe:O for:O use:O with:O the:O microwavawe:B-appliance <=> appropriate




how:O do:O i:O remove:O residue:B-removeobject from:O the:O oven:B-appliance 6553423rd:O <=> remove

how:O do:O i:O remove:O stains:B-removeobject from:O the:O microwavawe:B-appliance 6553423rd:O <=> remove

how:O can:O i:O remove:O a:O part:O of:O lights:B-removeobject from:O the:O oven:B-appliance <=> remove

how:O can:O i:O remove:O a:O part:O of:O residue:B-removeobject from:O the:O microwavawe:B-appliance <=> remove

how:O can:O i:O remove:O a:O part:O of:O door:B-removeobject from:O the:O oven:B-appliance <=> remove

how:O do:O i:O remove:O rack:B-removeobject from:O the:O oven:B-appliance <=> remove

how:O do:O i:O remove:O door:B-removeobject from:O the:O oven:B-appliance <=> remove

how:O do:O i:O remove:O rack:B-removeobject from:O the:O microwavawe:B-appliance <=> remove

is:O there:O any:O way:O to:O remove:O the:O rack:B-removeobject from:O the:O oven:B-appliance <=> remove

is:O there:O any:O way:O to:O remove:O the:O rack:B-removeobject from:O the:O microwavawe:B-applia


can:O you:O help:O me:O to:O reinstall:O rack:B-replaceobject from:O the:O oven:B-appliance <=> replace

can:O you:O help:O me:O to:O reinstall:O night:B-replaceobject light:I-replaceobject from:O the:O oven:B-appliance <=> replace

can:O you:O help:O me:O to:O reinstall:O door:B-replaceobject from:O the:O microwavawe:B-appliance <=> replace

how:O can:O i:O renew:O any:O of:O my:O cooktop:B-replaceobject oven:B-appliance <=> replace

how:O can:O i:O renew:O any:O of:O my:O drawer:B-replaceobject microwavawe:B-appliance <=> replace

how:O can:O i:O renew:O any:O of:O my:O cooktop:B-replaceobject oven:B-appliance <=> replace

the:O why:O how:O do:O i:O reinstall:O lights:B-replaceobject from:O the:O oven:B-appliance <=> replace

the:O why:O how:O do:O i:O reinstall:O rack:B-replaceobject from:O the:O oven:B-appliance <=> replace

the:O why:O how:O do:O i:O reinstall:O oven:B-replaceobject light:I-replaceobject from:O the:O oven:B-appliance <=> replace

can:O i:O reinstall:O cooktop:B-r

how:O to:O put:O on:O wifi:B-onofffeature microwavawe:B-appliance <=> on_off

how:O to:O put:O on:O wifi:B-onofffeature oven:B-appliance <=> on_off

how:O to:O put:O on:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature oven:B-appliance <=> on_off

can:O you:O please:O tell:O me:O how:O to:O use:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature on:O the:O microwavawe:B-appliance <=> on_off

can:O you:O please:O tell:O me:O how:O to:O use:O wifi:B-onofffeature on:O the:O microwavawe:B-appliance <=> on_off

can:O you:O please:O tell:O me:O how:O to:O use:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature on:O the:O oven:B-appliance <=> on_off

is:O this:O device:O capable:O of:O sound:B-onofffeature on:O any:O microwavawe:B-appliance types:O <=> on_off

is:O this:O device:O capable:O of:O sound:B-onofffeature on:O any:O oven:B-appliance types:O <=> on_off

is:O this:O device:O capable:O of:O wifi:B-onofffeature on:O any:O oven:B-appliance 

In [160]:
create_data_file(f_valid,df_valid)

is:O it:O safe:O to:O use:O paper:B-cookware bags:I-cookware in:O a:O oven:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O straw:B-cookware in:O a:O oven:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O straw:B-cookware in:O a:O oven:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O the:O recycled:B-cookware paper:I-cookware with:O the:O microwavawe:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O the:O metal:B-cookware with:O the:O oven:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O the:O polyester:B-cookware cup:I-cookware with:O the:O microwavawe:B-appliance <=> appropriate

will:O the:O ceramic:B-cookware be:O safe:O for:O use:O with:O the:O oven:B-appliance <=> appropriate

will:O the:O ceramic:B-cookware be:O safe:O for:O use:O with:O the:O oven:B-appliance <=> appropriate

will:O the:O browning:B-cookware plate:I-cookware be:O safe:O for:O use:O with:O the:O microwavawe:B-appliance <=> appropriate

what:O type:O of:O ceramic:B-cookware s


how:O are:O the:O cooktop:B-replaceobject replaced:O form:O the:O oven:B-appliance <=> replace

how:O are:O the:O lights:B-replaceobject replaced:O form:O the:O oven:B-appliance <=> replace

how:O are:O the:O door:B-replaceobject replaced:O form:O the:O microwavawe:B-appliance <=> replace

where:O do:O I:O replace:O rack:B-replaceobject from:O the:O microwavawe:B-appliance <=> replace

where:O do:O I:O replace:O rack:B-replaceobject from:O the:O microwavawe:B-appliance <=> replace

where:O do:O I:O replace:O night:B-replaceobject light:I-replaceobject from:O the:O microwavawe:B-appliance <=> replace

how:O do:O i:O use:O night:B-replaceobject light:I-replaceobject from:O the:O microwavawe:B-appliance <=> replace

how:O do:O i:O use:O drawer:B-replaceobject from:O the:O microwavawe:B-appliance <=> replace

how:O do:O i:O use:O charcoal:B-replaceobject filter:I-replaceobject from:O the:O oven:B-appliance <=> replace

how:O can:O i:O reinstall:O the:O door:B-replaceobject in:O the:O oven


what:O is:O the:O purpose:O of:O cook:B-purposefeature time:I-purposefeature in:O the:O oven:B-appliance <=> purpose

what:O is:O the:O purpose:O of:O cooktop:B-purposefeature lamp:I-purposefeature in:O the:O microwavawe:B-appliance <=> purpose

what:O is:O the:O purpose:O of:O air:B-purposefeature fry:I-purposefeature in:O the:O microwavawe:B-appliance <=> purpose

what:O is:O the:O purpose:O of:O the:O bread:B-purposefeature proof:I-purposefeature on:O the:O oven:B-appliance missing:O <=> purpose

what:O is:O the:O purpose:O of:O the:O oven:B-purposefeature clock:I-purposefeature on:O the:O microwavawe:B-appliance missing:O <=> purpose

what:O is:O the:O purpose:O of:O the:O steam:B-purposefeature clean:I-purposefeature on:O the:O oven:B-appliance missing:O <=> purpose

what:O is:O the:O purpose:O of:O display:B-purposefeature on:O the:O microwavawe:B-appliance <=> purpose

what:O is:O the:O purpose:O of:O broil:B-purposefeature on:O the:O microwavawe:B-appliance <=> purpose

what:O

In [161]:
# with open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance4\\train','w') as f_train:
#     for i in range(df_train.shape[0]):
#         for j in range(3):
#             slots = df_train.loc[i,'question_should_contain'].split(' ')
#             x = df_train.loc[i,'question']
#             x = re.sub(slots[0], random_slot(slots[0]), x)
#             x = re.sub(slots[1], random_slot(slots[1]), x)
#             x = x.lstrip()
#             x = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
#             x = ' '.join(x.split())
#             f_train.write(x + " <=> " + df_train.loc[i,'intent'] + "\n")
#             print(x + " <=> " + df_train.loc[i,'intent'] + "\n")
# f_train.close()

In [162]:
# with open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance3\\train','w') as f_train, open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance3\\test','w') as f_test, open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance3\\valid','w') as f_valid:
#     for i in range(data_trim.shape[0]):
#         for j in range(3):
#             slots = data_trim.loc[i,'question_should_contain'].split(' ')
#             x = data_trim.loc[i,'question']
#             x = re.sub(slots[0], random_slot(slots[0]), x)
#             x = re.sub(slots[1], random_slot(slots[1]), x)
#             x = x.lstrip()
#             x = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
#             #print(data_trim.loc[i,'intent'])
#             x = ' '.join(x.split())
#             if((i % 4 == 0) or (i % 4 == 1)):
#                 f_train.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
#                 print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
#             elif (i % 4 == 2):
#                 f_test.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
#                 print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
#             else:
#                 f_valid.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
#                 print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
# f_train.close()
# f_test.close()
# f_valid.close()